## Importation des bibliotheques

In [1]:
import numpy as np
import os
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from keras.models import Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split
import cv2
from tensorflow import keras

## Dossier dans laquel le dataset est situe

In [2]:
train_dir = '/home/pc/Documents/hand_gesture_cnn/Data/train_data'
test_dir = '/home/pc/Documents/hand_gesture_cnn/Data/test_train'
labels_dict = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,
                   'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,
                   'Z':25,'space':26,'del':27,'nothing':28}

In [3]:
def load_data():
    images = []
    labels = []
    size = (64,64)
    print("Importer le dossier : ",end = "")
    for folder in os.listdir(train_dir):
        print(folder, end = ' | ')
        for image in os.listdir(train_dir + "/" + folder):
            temp_img = cv2.imread(train_dir + '/' + folder + '/' + image)
            temp_img = cv2.resize(temp_img, size)
            images.append(temp_img)
            labels.append(labels_dict[folder])
    
    images = np.array(images)
    images = images.astype('float32')/255.0
    
    labels = keras.utils.to_categorical(labels,29)
    
    X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size = 0.05)
    
    print()
    print('Charger', len(X_train),'images pour training,','la taille de données de Train  =',X_train.shape)
    print('Charger', len(X_test),'images pour testing','la taille de données de Test =',X_test.shape)
    
    return X_train, X_test, Y_train, Y_test

In [4]:
%%time
X_train, X_test, Y_train, Y_test = load_data()

Importer le dossier : F | J | O | P | T | nothing | K | W | S | Z | del | E | M | L | C | A | V | D | B | R | U | H | X | N | Y | G | space | Q | I | 
Charger 82650 images pour training, la taille de données de Train  = (82650, 64, 64, 3)
Charger 4350 images pour testing la taille de données de Test = (4350, 64, 64, 3)
CPU times: user 2min 35s, sys: 7.77 s, total: 2min 43s
Wall time: 2min 44s


# Creation du modele

In [5]:
%%time
model = Sequential()
    
model.add(Conv2D(16, kernel_size = [3,3], padding = 'same', activation = 'relu', input_shape = (64,64,3)))
model.add(Conv2D(32, kernel_size = [3,3], padding = 'same', activation = 'relu'))
model.add(MaxPool2D(pool_size = [3,3]))
    
model.add(Conv2D(32, kernel_size = [3,3], padding = 'same', activation = 'relu'))
model.add(Conv2D(64, kernel_size = [3,3], padding = 'same', activation = 'relu'))
model.add(MaxPool2D(pool_size = [3,3]))
    
model.add(Conv2D(128, kernel_size = [3,3], padding = 'same', activation = 'relu'))
model.add(Conv2D(256, kernel_size = [3,3], padding = 'same', activation = 'relu'))
model.add(MaxPool2D(pool_size = [3,3]))
    
model.add(BatchNormalization())
    
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation = 'relu', kernel_regularizer = regularizers.l2(0.001)))
model.add(Dense(29, activation = 'softmax'))
    
model.compile(optimizer = 'adam', loss = keras.losses.categorical_crossentropy, metrics = ["accuracy"])
    
print("Modele Créer")
model.summary()

Modele Créer
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 21, 21, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 32)        9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                       

In [6]:
%%time
model.fit(X_train, Y_train, batch_size = 64, epochs = 5, validation_split = 0.1)

Epoch 1/5
1163/1163 [==============================] - 873s 749ms/step - loss: 0.8447 - accuracy: 0.8186 - val_loss: 0.3944 - val_accuracy: 0.9195
Epoch 2/5
1163/1163 [==============================] - 970s 834ms/step - loss: 0.1906 - accuracy: 0.9772 - val_loss: 0.2094 - val_accuracy: 0.9667
Epoch 3/5
1163/1163 [==============================] - 915s 787ms/step - loss: 0.1654 - accuracy: 0.9823 - val_loss: 0.1584 - val_accuracy: 0.9802
Epoch 4/5
1163/1163 [==============================] - 914s 786ms/step - loss: 0.1554 - accuracy: 0.9851 - val_loss: 0.1063 - val_accuracy: 0.9940
Epoch 5/5
1163/1163 [==============================] - 1040s 895ms/step - loss: 0.1471 - accuracy: 0.9858 - val_loss: 0.1735 - val_accuracy: 0.9864
CPU times: user 7h 36min 8s, sys: 14min 2s, total: 7h 50min 11s
Wall time: 1h 18min 37s


In [10]:
model.save("model_ASL.h5")